# **CAP 5404 Deep Learning for Computer Graphics**
# *Project II. Neural Networks & Computer Graphics*

Pranath Reddy Kumbam (**UFID**: 8512-0977)



## Part 2: Regression

### Load Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path to Working Directory 
%cd drive/My Drive/Acad/DLCG/Project2

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from torchvision import models
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
# Mean Chrominance values
def get_mean_chrominance(img1, img2):
  return (np.mean(img1), np.mean(img2))

# Min-Max Norm 
def norm(images):
    data = []
    for sample in images:
      min = np.amin(sample)
      max = np.amax(sample)
      range_val = max - min
      sample = (sample-min)/range_val
      sample = sample.reshape(1,128,128)
      data.append(sample)
    data = np.asarray(data)
    return data

# Import Data
l_train = norm(np.load('./Data/arrays/Faces/L_train.npy'))
a_train = np.load('./Data/arrays/Faces/a_train.npy')/255
b_train = np.load('./Data/arrays/Faces/b_train.npy')/255

l_test = norm(np.load('./Data/arrays/Faces/L_test.npy'))
a_test = np.load('./Data/arrays/Faces/a_test.npy')/255
b_test = np.load('./Data/arrays/Faces/b_test.npy')/255

x_train = l_train
y_train = np.array([get_mean_chrominance(a_train[x], b_train[x]) for x in range(x_train.shape[0])])
y_ts = np.array([get_mean_chrominance(a_test[x], b_test[x]) for x in range(l_test.shape[0])])

x_val = l_test[int(l_test.shape[0]*0.5):]
y_val = y_ts[int(l_test.shape[0]*0.5):]
x_test = l_test[:int(l_test.shape[0]*0.5)]
y_test = y_ts[:int(l_test.shape[0]*0.5)]

batch_size = 100
# Shuffle Data
x_train, y_train = shuffle(x_train, y_train, random_state=0)
x_test, y_test = shuffle(x_test, y_test, random_state=0)
x_val, y_val = shuffle(x_val, y_val, random_state=0)

# Split into batches
batch_size = 100
a = 0
b = batch_size
data_temp = []
data_temp2 = []
for i in range(int(x_train.shape[0]/batch_size)):
    data_temp.append(x_train[a:b])
    data_temp2.append(y_train[a:b])
    a += batch_size
    b += batch_size
x_train = np.asarray(data_temp)
y_train = np.asarray(data_temp2)

# Print data shape
print("Data Shape")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_val.shape)
print(y_val.shape)

### Define Models

In [ ]:
# As described in the project description 
# A simple CNN model with seven Conv Blocks and three Feature maps for hidden layers
class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()

        self.regressor = nn.Sequential(
            nn.Conv2d(1, 3, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(3, 2, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

# Hyperparameter tuning
# Trying different set of feature maps 
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()

        self.regressor = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(256, 64),
            nn.Linear(64, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

# Increasing the number of feature maps 
class CNN3(nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()

        self.regressor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.Linear(128, 2),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.regressor(x)
        return x

# Push model to gpu
model = CNN2().to("cuda")

### Train Model

In [ ]:
# Reset Model
for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

# Loss Function
criteria = torch.nn.MSELoss()

# Hyperparameter tuning - Grid search on LR and epochs
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
n_epochs = 100

# Training
loss_array = []
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:
    train_loss = 0.0
    
    for i in range(x_train.shape[0]):

        data = torch.from_numpy(x_train[i].astype('float32'))
        if torch.cuda.is_available():
          data = data.cuda()
        labels = torch.tensor(y_train[i], dtype=torch.float, device="cuda")
        optimizer.zero_grad()
        outputs = model(data)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    # Monitor Validation Loss
    '''
    val_data = torch.from_numpy(x_val.astype('float32'))
    if torch.cuda.is_available():
      val_data = val_data.cuda()
    val_labels = torch.tensor(y_val, dtype=torch.float, device="cuda")
    val_outputs = model(val_data)
    val_loss = criteria(val_outputs, val_labels)
    '''

    train_loss_avg = train_loss/x_train.shape[0]
    #pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy(), 'Val Loss': val_loss.detach().cpu().numpy() })  
    pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy() })  

# Validation Result for Hyperparameter Search
val_data = torch.from_numpy(x_val.astype('float32'))
if torch.cuda.is_available():
  val_data = val_data.cuda()
val_labels = torch.tensor(y_val, dtype=torch.float, device="cuda")
val_outputs = model(val_data)
val_loss = criteria(val_outputs, val_labels)
loss = mean_squared_error(val_outputs.detach().cpu().numpy(), val_labels.cpu().numpy())
print("Val Result: " + str(loss))

### Save Best Model and Test 

In [ ]:
# Reset Model
for layer in model.children():
   if hasattr(layer, 'reset_parameters'):
       layer.reset_parameters()

# Model
model = CNN2().to("cuda")

# Loss Function
criteria = torch.nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
n_epochs = 500

# Training
loss_array = []
pbar = tqdm(range(1, n_epochs+1))
for epoch in pbar:
    train_loss = 0.0
    
    for i in range(x_train.shape[0]):

        data = torch.from_numpy(x_train[i].astype('float32'))
        if torch.cuda.is_available():
          data = data.cuda()
        labels = torch.tensor(y_train[i], dtype=torch.float, device="cuda")
        optimizer.zero_grad()
        outputs = model(data)
        loss = criteria(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss

    train_loss_avg = train_loss/x_train.shape[0]
    loss_array.append(train_loss_avg.detach().cpu().numpy())
    pbar.set_postfix({ 'Training Loss': train_loss_avg.detach().cpu().numpy() })  

# Export Training Loss for Plot
np.save('./Out/CNN_Regressor_train_loss_Faces_ReLU.npy', loss_array)

# Export Trained Model for Transfer Learning
torch.save(model, './Out/CNN_Regressor_Faces_ReLU.pth')

# Testing on Test Data
test_data = torch.from_numpy(x_test.astype('float32'))
if torch.cuda.is_available():
  test_data = test_data.cuda()
test_labels = torch.tensor(y_test, dtype=torch.float, device="cuda")
test_outputs = model(test_data)
a_loss = mean_squared_error(test_outputs[:, 0].detach().cpu().numpy(), test_labels[:, 0].cpu().numpy())
b_loss = mean_squared_error(test_outputs[:, 1].detach().cpu().numpy(), test_labels[:, 1].cpu().numpy())
print("Test a Result: " + str(a_loss))
print("Test b Result: " + str(b_loss))